In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [2]:
tabla_completa = pd.read_csv('events_up_to_01062018.csv');
res = pd.read_csv('labels_training_set.csv');

C:\Users\Santi\AppData\Roaming\Python\Python36\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tabla_completa['aux']=1
#Voy a contar la cantidad para cada tipo de evento de cada persona
num_eventos = tabla_completa.groupby(['person','event'])['aux'].sum()

num_eventos = num_eventos.unstack().reset_index().fillna(0)
num_eventos = num_eventos.set_index('person')
num_eventos['total_eventos'] = num_eventos.sum(axis=1)
#normalizo los numeros obtenidos según la cantidad de eventos totales de esa persona
num_eventos_norm = num_eventos.loc[:,"ad campaign hit":"visited site"].div(num_eventos["total_eventos"], axis=0)
#num_eventos.sort_values(by='conversion',ascending=False)
#num_eventos_norm.sort_values(by='conversion',ascending=False)


In [4]:
num_eventos_norm = num_eventos_norm.reset_index()
num_eventos = num_eventos.reset_index()
total_eventos = pd.merge(num_eventos,num_eventos_norm, on='person',how='inner')
total_eventos.head()

event,person,ad campaign hit_x,brand listing_x,checkout_x,conversion_x,generic listing_x,lead_x,search engine hit_x,searched products_x,staticpage_x,...,brand listing_y,checkout_y,conversion_y,generic listing_y,lead_y,search engine hit_y,searched products_y,staticpage_y,viewed product_y,visited site_y
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.500000,0.000000,0.166667,0.0,0.000000,0.000000,0.0,0.000000,0.333333
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.055804,0.004464,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.830357,0.075893
2,00091a7a,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.300000,0.100000
3,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,...,0.116505,0.029126,0.004854,0.067961,0.0,0.004854,0.000000,0.0,0.742718,0.029126
4,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,...,0.000000,0.058824,0.000000,0.058824,0.0,0.058824,0.529412,0.0,0.176471,0.058824


In [5]:
intento = total_eventos.drop(['conversion_x','conversion_y'],axis='columns')
intento_f = pd.merge(intento,res,on='person',how='inner')

intento_f = intento_f.set_index('person')

In [6]:
final = pd.read_csv('trocafone_kaggle_test.csv')
final = pd.merge(intento,final,on='person',how='inner')
final = final.set_index('person')
x_entrega = final

In [7]:
#XGBoost

X, y = intento_f.iloc[:,:-1],intento_f.iloc[:,-1]
data_dmatrix = xgb.DMatrix(data=X,label=y)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 7, n_estimators = 10)

xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=7, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.264017


In [10]:
entrega = xg_reg.predict(x_entrega)
y_summit = pd.DataFrame(entrega)
y_summit.columns = ['label']
y_summit['person'] = x_entrega.reset_index()['person']
y_person = x_entrega.reset_index()['person']

y_summit = y_summit[['person','label']]
y_summit.to_csv('summit06.csv', index=False)

In [11]:
y_summit.max()

person    fffd1246
label     0.436373
dtype: object